# Ski resorts Dashboard

Dasboard which to visualize information about ski resorts across the world

It's actually deployed at the address below :
http://mat148.pythonanywhere.com/




In [114]:
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate
from dash_bootstrap_templates import load_figure_template

import plotly.express as px
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

resorts = (
    pd.read_csv("../Data/Ski Resorts/resorts.csv", encoding = "ISO-8859-1")
    .assign(
        country_elevation_rank = lambda x: x.groupby("Country", as_index=False)["Highest point"].rank(ascending=False),
        country_price_rank = lambda x: x.groupby("Country", as_index=False)["Price"].rank(ascending=False),
        country_slope_rank = lambda x: x.groupby("Country", as_index=False)["Total slopes"].rank(ascending=False),
        country_cannon_rank = lambda x: x.groupby("Country", as_index=False)["Snow cannons"].rank(ascending=False),
    ))

dict_pays = {cont : list(resorts[resorts["Continent"]==cont]["Country"].unique()) for cont in resorts["Continent"].unique() }

In [3]:
resorts.head()

,ID,Resort,Latitude,Longitude,Country,Continent,Price,Season,Highest point,Lowest point,Beginner slopes,Intermediate slopes,Difficult slopes,Total slopes,Longest run,Snow cannons,Surface lifts,Chair lifts,Gondola lifts,Total lifts,Lift capacity,Child friendly,Snowparks,Nightskiing,Summer skiing,country_elevation_rank,country_price_rank,country_slope_rank,country_cannon_rank
0,1,Hemsedal,60.928244,8.383487,Norway,Europe,46,November - May,1450,620,29,10,4,43,6,325,15,6,0,21,22921,Yes,Yes,Yes,No,2.0,1.5,4.0,2.0
1,2,Geilosiden Geilo,60.534526,8.206372,Norway,Europe,44,November - April,1178,800,18,12,4,34,2,100,18,6,0,24,14225,Yes,Yes,Yes,No,4.0,6.0,6.5,4.0
2,3,Golm,47.057810,9.828167,Austria,Europe,48,December - April,2110,650,13,12,1,26,9,123,4,4,3,11,16240,Yes,No,No,No,37.0,31.5,63.0,45.0
3,4,Red Mountain Resort-Rossland,49.105520,-117.846280,Canada,North America,60,December - April,2075,1185,20,50,50,120,7,0,2,5,1,8,9200,Yes,Yes,Yes,No,13.0,13.0,5.0,15.0
4,5,Hafjell,61.230369,10.529014,Norway,Europe,45,November - April,1030,195,33,7,4,44,6,150,14,3,1,18,21060,Yes,Yes,Yes,No,9.0,3.5,3.0,3.0


In [330]:
#%%writefile app.py

dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.MORPH, dbc_css])

load_figure_template("MORPH")

app.layout = html.Div([
    dcc.Tabs(className="dbc", children=[dcc.Tab([
    dbc.Row(html.H2(id="title_1", style={"text-align":"center","fontSize":24})),
    dbc.Row([dbc.Col(dbc.Card([
        html.H6("Price Limit", style={"fontWeight":"bold"}),
        dcc.Slider(
        id="price",
        min=0,
        max=150,
        step=25,
        value=150,
        className="dbc"
),
    html.Br(),
    html.H6("Features Preferences", style={"fontWeight":"bold"}),
    dcc.Checklist(
        id="summer",
        options=[{"label":"With Summer Skiing", "value":"Yes"}],
        style={"fontSize":14},
        value=[],
        className="dbc"
),
    dcc.Checklist(
        id="night",
        options=[{"label":"With Nightskiing", "value":"Yes"}],
        style={"fontSize":14},
        value=[],
        className="dbc"
),
    dcc.Checklist(
        id="snow",
        options=[{"label":"With Snowparks", "value":"Yes"}],
        style={"fontSize":14},
        value=[],
        className="dbc"
)
    ])),
    dbc.Col(dcc.Graph(id="map",className="dbc"), width=9)
    ])
    ], label="Resort Map"),
    
    dcc.Tab([
    
        dbc.Row(html.H2(id="title_2",style={"text-align":"center","fontSize":30})),
        dbc.Row([
        html.Br(),
        html.H6("Select a Continent"),
        
        dbc.Col([dcc.Dropdown(id="continent",
                     options=resorts.Continent.unique(),
                     value="Europe",
                     className="dbc"
                    ),
                 html.Br(),
                 html.H6("Select a Country"),
                
                 dcc.Dropdown(id="pays",
                     value="Norway",
                     className="dbc"
                    ),
                html.Br(),
                 html.H6("Select a Metric to Plot"),
                 dcc.Dropdown(id="metric",
                     options=[{"label":"Elevation", "value":"Highest point"},
                             {"label":"Price", "value":"Price"},
                             {"label":"Slope", "value":"Total slopes"},
                             {"label":"Cannon", "value":"Snow cannons"}],
                     value="Price",
                     className="dbc")
        
                ]),
        
        dbc.Col(dcc.Graph(id="bar",hoverData={'points': [{'customdata' : ['Hemsedal']}]}), width=7),
        dbc.Col([
            html.H5("Resort Report Card"),
            dbc.Row(dbc.Card(id="resort", style={"text-align":"center", "fontSize":15})),
            dbc.Row([dbc.Col(
                dbc.Card(id="elevation_rank", style={"text-align":"left", "fontSize":12})
            ),
                dbc.Col(dbc.Card(id="price_rank", style={"text-align":"left", "fontSize":12})
                       )
            
                    ]),
            dbc.Row([dbc.Col(
                dbc.Card(id="slope_rank", style={"text-align":"left", "fontSize":12})
            ),
                dbc.Col(dbc.Card(id="cannon_rank", style={"text-align":"left", "fontSize":12})
                       )
            
                    ])
        ])
        
        
            ])   
            
    ],label="Country Profiler")
              
        
    ])
    
])




@app.callback(Output("pays", "options"), Input("continent", "value"))
def drop(continent):
    return dict_pays[continent]


@app.callback(Output("bar", "figure"), Output("title_2", "children"), Input("metric", "value"), Input("pays", "value"))
def drop(metric, pays):
    resorts_map = resorts[resorts["Country"]==pays].sort_values(metric, ascending=False).iloc[:10]
    fig = px.bar(resorts_map,
                 x="Resort",
                 y=metric,
                 hover_name="Resort",
                 custom_data=["Resort"],
                 height=550,
                 width=750
                )
    fig.update_xaxes(showticklabels=False),
    fig.update_yaxes(showgrid =False),
    fig.update_layout(margin={"r":0,"t":0,"l":20,"b":20})
    
    title = f"Top Resorts in {pays} by {metric}"
    return fig, title
        
    
@app.callback(Output("resort", "children"), Output("elevation_rank", "children"), Output("price_rank", "children"), Output("slope_rank", "children"), Output("cannon_rank", "children"), Input("bar","hoverData"))
def drop(resort):
    
    resort_name = resort["points"][0]["customdata"][0]
    
    
    elevation = f"Elevation Rank : {int(resorts[resorts['Resort']==resort_name]['country_elevation_rank'].iloc[0])}"
    price = f"Price Rank : {int(resorts[resorts['Resort']==resort_name]['country_price_rank'].iloc[0])}"
    slope = f"Slope Rank : {int(resorts[resorts['Resort']==resort_name]['country_slope_rank'].iloc[0])}"
    cannon = f"Cannon Rank : {int(resorts[resorts['Resort']==resort_name]['country_cannon_rank'].iloc[0])}"

    
    return resort_name, elevation, price, slope, cannon

#    price = f"Price Rank : {int(resorts[resorts['Resort']==resort]['country_price_rank'][0])}"
    


@app.callback(Output("map", "figure"),Output("title_1", "children"), Input("price", "value"), Input("summer", "value"), Input("night", "value"), Input("snow", "value"))
def graph(price, summer, night, snow):
    
    resort_map = resorts[resorts["Price"]<=price]
    
    if len(summer)>0:
        resort_map = resort_map[resort_map["Summer skiing"]=="Yes"]
    if len(night)>0:
        resort_map = resort_map[resort_map["Nightskiing"]=="Yes"]
    if len(snow)>0:
        resort_map = resort_map[resort_map["Snowparks"]=="Yes"]
    
    figure = px.density_mapbox(
        resort_map,
        lat="Latitude",
        lon="Longitude",
        z="Total slopes",
        mapbox_style="open-street-map",
        hover_name = "Country",
        hover_data =["Resort"],
        color_continuous_scale="blues",
        zoom=2.5,
        height=600,
        width=800
    )
    figure.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    
    title = f"Resorts with a ticket price less than {price}$"
    
    return figure, title

if __name__ == "__main__":
    app.run_server(port="8055")

Dash app running on http://127.0.0.1:8055/
